In [1]:
import time

import numpy as np
import pandas as pd

from pyhive import hive
from tqdm.notebook import tqdm

In [2]:
conn = hive.connect("localhost")
cursor = conn.cursor()

In [3]:
# List tables
cursor.execute("SHOW TABLES")
cursor.fetchall()

[('geolocation',), ('titles',)]

In [4]:
def create_table():
    query = """
        CREATE TABLE titles (
            id INTEGER,
            show_id STRING,
            type STRING,
            title STRING,
            director STRING,
            actor STRING,
            country STRING,
            date_added DATE,
            release_year SMALLINT,
            rating STRING,
            duration STRING,
            duration_min SMALLINT,
            listed_in STRING,
            description STRING
        )
        PARTITIONED BY (platform STRING)
        ROW FORMAT DELIMITED
        FIELDS TERMINATED BY ','
        STORED AS ORC
        LOCATION '/user/root/titles'
        TBLPROPERTIES ('transactional'='true')
    """

    cursor = conn.cursor()
    cursor.execute(query)

# create_table()

In [5]:
def test_insert():
    query = """
        INSERT INTO titles
        VALUES (1, 'AMZN-1', 'Movie', 'Title', 'Director', 'Actor', 'MY', '2022-03-01', 2022, '18+', '111 min', 111, 'Comedy', 'Description', 'amazon_prime')
    """

    cursor = conn.cursor()
    cursor.execute(query)

    cursor.execute("SELECT * FROM titles")
    print(cursor.fetchall())

# test_insert()

In [6]:
def test_insert_with_null():
    query = """
        INSERT INTO titles
         VALUES (1, 'AMZN-1', 'Movie', NULL, 'Director', 'Actor', 'MY', '2022-03-01', 2022, '18+', '111 min', 111, 'Comedy', 'Description', 'amazon_prime')
    """

    cursor = conn.cursor()
    cursor.execute(query)

    cursor.execute("SELECT * FROM titles WHERE title IS NULL")
    print(cursor.fetchall())

# test_insert_with_null()

In [7]:
def test_clean_up():
    cursor = conn.cursor()
    cursor.execute("DELETE FROM titles")

    cursor.execute("SELECT * FROM titles")
    print(cursor.fetchall())

# test_clean_up()

In [8]:
df = pd.read_csv("datasets/union.csv")
df.replace(np.nan, None, inplace=True)

In [9]:
def format_string(string):
    return "'" + string.replace("'", "\\'") + "'"


def load_data(batch_size=1000):
    cursor = conn.cursor()
    key = 1
    batches = np.array_split(df, len(df) // batch_size + 1)

    for batch in tqdm(batches):
        tuples_batch = []

        for idx, row in batch.iterrows():
            show_id = format_string(row.show_id)
            type = format_string(row.type)
            title = format_string(row.title)
            listed_in = format_string(row.listed_in)
            platform = format_string(row.platform)

            if row.director is None:
                director = "NULL"
            else:
                director = format_string(row.director)

            if row.cast is None:
                cast = "NULL"
            else:
                cast = format_string(row.cast)

            if row.country is None:
                country = "NULL"
            else:
                country = format_string(row.country)

            if pd.isnull(row.date_added):
                date = "NULL"
            else:
                date = format_string(row.date_added)

            if row.rating is None:
                rating = "NULL"
            else:
                rating = format_string(row.rating)

            if row.duration is None:
                duration = "NULL"
            else:
                duration = format_string(row.duration)

            if row.duration_min is None:
                duration_min = "NULL"
            else:
                duration_min = str(row.duration_min)

            if row.description is None:
                description = "NULL"
            else:
                description = format_string(row.description)

            tuple = (
                str(key), show_id, type, title, director, cast, country, date,
                str(row.release_year), rating, duration, duration_min, listed_in,
                description, platform
            )

            tuple_str = f"({', '.join(tuple)})"

            tuples_batch.append(tuple_str)
            key += 1

        query = f"INSERT INTO titles VALUES {', '.join(tuples_batch)}"
        cursor.execute(query)

# load_data()

In [10]:
def moviews_and_tv_shows():
    query = """
        SELECT type, COUNT(DISTINCT show_id)
        FROM titles
        GROUP BY type
    """

    cursor.execute(query)

    return cursor.fetchall()

start = time.time_ns()
results = moviews_and_tv_shows()
print("Execution time (ns):", time.time_ns() - start)

df_1 = pd.DataFrame(results, columns=["type", "count"])
df_1

Execution time (ns): 3104890300


,type,count
0,Movie,16481
1,TV Show,6517


In [11]:
def movies_and_tv_shows_by_platform():
    query = """
        SELECT platform, type, COUNT(DISTINCT show_id)
        FROM titles
        GROUP BY platform, type
        ORDER BY platform, type
    """

    cursor.execute(query)

    return cursor.fetchall()

start = time.time_ns()
results = movies_and_tv_shows_by_platform()
print("Execution time (ns):", time.time_ns() - start)

df_2 = pd.DataFrame(results, columns=["platform", "type", "count"])
df_2

Execution time (ns): 4459195000


,platform,type,count
0,amazon_prime,Movie,7814
1,amazon_prime,TV Show,1854
2,disney,Movie,1052
3,disney,TV Show,398
4,hulu,Movie,1484
5,hulu,TV Show,1589
6,netflix,Movie,6131
7,netflix,TV Show,2676


In [12]:
def movies_and_tv_shows_trend():
    query = """
        SELECT type, release_year, COUNT(DISTINCT show_id)
        FROM titles
        GROUP BY type, release_year
        ORDER BY type, release_year
    """

    cursor.execute(query)

    return cursor.fetchall()

start = time.time_ns()
results = movies_and_tv_shows_trend()
print("Execution time (ns):", time.time_ns() - start)

df_3 = pd.DataFrame(results, columns=["type", "release_year", "count"])
df_3

Execution time (ns): 4195411700


,type,release_year,count
0,Movie,1920,3
1,Movie,1922,2
2,Movie,1923,2
3,Movie,1924,1
4,Movie,1925,8
...,...,...,...
169,TV Show,2017,571
170,TV Show,2018,711
171,TV Show,2019,808
172,TV Show,2020,853


In [13]:
def movies_and_tv_shows_trend_by_platform():
    query = """
        SELECT platform, type, release_year, COUNT(DISTINCT show_id)
        FROM titles
        GROUP BY platform, type, release_year
        ORDER BY platform, type, release_year
    """

    cursor.execute(query)

    return cursor.fetchall()

start = time.time_ns()
results = movies_and_tv_shows_trend_by_platform()
print("Execution time (ns):", time.time_ns() - start)

df_4 = pd.DataFrame(results, columns=["platform", "type", "release_year", "count"])
df_4

Execution time (ns): 4458247000


,platform,type,release_year,count
0,amazon_prime,Movie,1920,3
1,amazon_prime,Movie,1922,2
2,amazon_prime,Movie,1923,1
3,amazon_prime,Movie,1924,1
4,amazon_prime,Movie,1925,8
...,...,...,...,...
531,netflix,TV Show,2017,265
532,netflix,TV Show,2018,380
533,netflix,TV Show,2019,397
534,netflix,TV Show,2020,436


In [14]:
def movies_and_tv_shows_by_rating():
    query = """
        SELECT rating, COUNT(DISTINCT show_id) AS count
        FROM titles
        WHERE rating IS NOT NULL
            AND rating NOT LIKE '%min'
            AND rating NOT LIKE '%Season'
            AND rating NOT LIKE '%Seasons'
        GROUP BY rating
        ORDER BY count DESC
    """

    cursor.execute(query)

    return cursor.fetchall()

start = time.time_ns()
results = movies_and_tv_shows_by_rating()
print("Execution time (ns):", time.time_ns() - start)

df_5 = pd.DataFrame(results, columns=["rating", "count"])
df_5

Execution time (ns): 4570036300


,rating,count
0,TV-MA,3675
1,TV-14,3138
2,R,2154
3,13+,2117
4,TV-PG,1654
5,16+,1547
6,ALL,1268
7,18+,1243
8,PG-13,1112
9,PG,881


In [15]:
def movies_and_tv_shows_by_duration():
    query = """
        SELECT
            CASE
                WHEN duration_min >= 0 AND duration_min <= 30 THEN 'Below 30 minutes'
                WHEN duration_min > 30 AND duration_min <= 60 THEN '30 - 60 minutes'
                WHEN duration_min > 60 AND duration_min <= 120 THEN '1 - 2 hours'
                WHEN duration_min > 120 then 'Above 2 hours'
            END AS duration,
            COUNT(DISTINCT show_id) AS count
        FROM titles
        WHERE duration_min IS NOT NULL
        GROUP BY
            CASE
                WHEN duration_min >= 0 AND duration_min <= 30 THEN 'Below 30 minutes'
                WHEN duration_min > 30 AND duration_min <= 60 THEN '30 - 60 minutes'
                WHEN duration_min > 60 AND duration_min <= 120 THEN '1 - 2 hours'
                WHEN duration_min > 120 then 'Above 2 hours'
            END
    """

    cursor.execute(query)

    return cursor.fetchall()

start = time.time_ns()
results = movies_and_tv_shows_by_duration()
print("Execution time (ns):", time.time_ns() - start)

df_6 = pd.DataFrame(results, columns=["duration", "count"])
df_6

Execution time (ns): 2862582000


,duration,count
0,1 - 2 hours,11318
1,30 - 60 minutes,1468
2,Above 2 hours,2434
3,Below 30 minutes,779


In [16]:
def top_genres_by_platform_and_shows():
    query = """
        WITH base_table AS (
            SELECT platform, listed_in, COUNT(DISTINCT show_id) AS count
            FROM titles
            GROUP BY platform, listed_in
        ),
        group_ranked_table AS (
            SELECT
                platform,
                listed_in,
                count,
                ROW_NUMBER() OVER (PARTITION BY platform ORDER BY count DESC) AS rank
            FROM base_table
        )
        SELECT platform, listed_in, count
        FROM group_ranked_table
        WHERE rank <= 10
    """

    cursor.execute(query)

    return cursor.fetchall()

start = time.time_ns()
results = top_genres_by_platform_and_shows()
print("Execution time (ns):", time.time_ns() - start)

df_7 = pd.DataFrame(results, columns=["platform", "listed_in", "count"])
df_7

Execution time (ns): 4437007800


,platform,listed_in,count
0,amazon_prime,Drama,3687
1,amazon_prime,Comedy,2099
2,amazon_prime,Action,1657
3,amazon_prime,Suspense,1501
4,amazon_prime,Kids,1085
5,amazon_prime,Documentary,993
6,amazon_prime,Special Interest,980
7,amazon_prime,Horror,875
8,amazon_prime,Romance,674
9,amazon_prime,Animation,547


In [17]:
def top_countries_by_shows():
    query = """
        SELECT country, COUNT(DISTINCT show_id) AS count
        FROM titles
        WHERE country IS NOT NULL
        GROUP BY country
        ORDER BY count DESC
        LIMIT 10
    """

    cursor.execute(query)
    
    return cursor.fetchall()

start = time.time_ns()
results = top_countries_by_shows()
print("Execution time (ns):", time.time_ns() - start)

df_8 = pd.DataFrame(results, columns=["country", "count"])
df_8

Execution time (ns): 4368198200


,country,count
0,United States,6304
1,India,1299
2,United Kingdom,1172
3,Canada,645
4,Japan,615
5,France,475
6,Germany,293
7,South Korea,260
8,Spain,258
9,Australia,225


In [18]:
def top_directors_by_shows():
    query = """
        SELECT director, COUNT(DISTINCT show_id) AS count
        FROM titles
        WHERE director IS NOT NULL
        GROUP BY director
        ORDER BY count DESC
        LIMIT 10
    """

    cursor.execute(query)

    return cursor.fetchall()

start = time.time_ns()
results = top_directors_by_shows()
print("Execution time (ns):", time.time_ns() - start)

df_9 = pd.DataFrame(results, columns=["director", "count"])
df_9

Execution time (ns): 4320654300


,director,count
0,Mark Knight,113
1,Cannis Holder,61
2,Jay Chapman,46
3,Moonbug Entertainment,37
4,Arthur van Merwijk,30
5,Manny Rodriguez,27
6,Jay Karas,22
7,John English,20
8,Rajiv Chilaka,19
9,"Raúl Campos, Jan Suter",18


In [19]:
def top_actors_by_shows():
    query = """
        SELECT actor, COUNT(DISTINCT show_id) AS count
        FROM titles
        WHERE actor IS NOT NULL
        GROUP BY actor
        ORDER BY count DESC
        LIMIT 10
    """

    cursor.execute(query)

    return cursor.fetchall()

start = time.time_ns()
results = top_actors_by_shows()
print("Execution time (ns):", time.time_ns() - start)

df_10 = pd.DataFrame(results, columns=["actor", "count"])
df_10

Execution time (ns): 5271187100


,actor,count
0,Anupam Kher,60
1,Maggie Binkley,56
2,Amitabh Bachchan,47
3,Shah Rukh Khan,46
4,Jim Cummings,44
5,Nassar,43
6,Akshay Kumar,41
7,Paresh Rawal,39
8,Naseeruddin Shah,39
9,Danny Trejo,39
